In [2]:

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import itertools

# Setting data

In [10]:

df = pd.read_excel('basket_abcde2.xlsx')
df_graph = pd.read_excel('basket_abcde.xlsx')
#df = df.drop(columns = ['e'])

In [4]:
df = df.rename(columns={'Unnamed: 0':'invoiceid'})
df = df.set_index('invoiceid')
df

,a,b,c,d,e
invoiceid,,,,,
1,0,1,0,0,0
2,0,1,0,0,0
3,0,1,0,1,0
4,0,1,0,1,0
5,0,1,0,1,0
6,0,1,0,0,0
7,0,1,0,0,0
8,0,1,1,0,1
9,0,1,1,0,1


In [5]:
item_list = list(df.columns)
print(item_list)

['a', 'b', 'c', 'd', 'e']


In [6]:
df_graph = df_graph.rename(columns={'Unnamed: 0':'invoiceid'})
df_graph = df_graph.set_index('invoiceid')

In [7]:
df['a'] = df['a']*2
df['b'] = df['b']*3
df['c'] = df['c']*1
df['d'] = df['d']*2
df['e'] = df['e']*4


In [7]:
quantity = df[df!=0].count()

# Graph

## Finding subgraphs using the binary valued basket

In [8]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [9]:
df_graph

,a,b,c,d,e
invoiceid,,,,,
1,0,1,0,0,0
2,0,1,0,0,0
3,0,1,0,0,0
4,0,1,0,0,0
5,0,1,0,0,0
6,0,1,0,0,0
7,0,1,0,0,0
8,0,1,0,0,0
9,0,1,0,0,0


In [10]:
def adjacency_from_dataframe(df):
    a = df.to_numpy()
    m = a.shape[1]
    adj = np.empty((m, m), dtype=np.bool)
    for j in range(m):
        col = a[:, j]
        rows = a[col == 1]
        adj[j] = np.logical_or.reduce(rows)
    return adj

In [11]:
items = item_list
adj = adjacency_from_dataframe(df_graph)

g = nx.from_numpy_matrix(adj)

In [12]:
g_df = nx.to_pandas_adjacency(g)
g_df

,0,1,2,3,4
0,1.0,1.0,1.0,0.0,0.0
1,1.0,1.0,1.0,0.0,0.0
2,1.0,1.0,1.0,0.0,0.0
3,0.0,0.0,0.0,1.0,1.0
4,0.0,0.0,0.0,1.0,1.0


In [13]:
components = [g.subgraph(c).copy() for c in nx.connected_components(g)]
for idx,g in enumerate(components,start=1):
    print(f"Component {idx}: Nodes: {g.nodes()} Edges: {g.edges()}")
    

Component 1: Nodes: [0, 1, 2] Edges: [(0, 0), (0, 1), (0, 2), (1, 1), (1, 2), (2, 2)]
Component 2: Nodes: [3, 4] Edges: [(3, 3), (3, 4), (4, 4)]


In [14]:
list(components[0].edges())[0]

list(g.edges())[0]

(3, 3)

In [15]:
for g in components:
    print("subgraph")
    for edge in list(g.edges()):
        if(item_list[edge[0]]!= item_list[edge[1]]):
            print(item_list[edge[0]], item_list[edge[1]])

subgraph
a b
a c
b c
subgraph
d e


In [16]:
item_list.sort()
item_list

['a', 'b', 'c', 'd', 'e']

# Characteristic function

## 1. Subset creation
## 2. Characteristic function calculation for each subset

In [17]:
# Finding invoice wise sum

basket = df
invoice_sum = list(basket.sum(axis = 1))
invoice_sum

[3, 3, 5, 5, 5, 3, 3, 8, 8, 10, 5, 5, 6, 8, 9, 9, 7, 7, 7, 7]

In [8]:
df

,a,b,c,d,e
invoiceid,,,,,
1,0,3,0,0,0
2,0,3,0,0,0
3,0,3,0,2,0
4,0,3,0,2,0
5,0,3,0,2,0
6,0,3,0,0,0
7,0,3,0,0,0
8,0,3,1,0,4
9,0,3,1,0,4


In [18]:
basket[basket!=0].count()

a     7
b    12
c     9
d    10
e    11
dtype: int64

In [19]:
print('itemwise sum: \n',basket.sum())
print('total sum: ', basket.sum().sum())

itemwise sum: 
 a    14
b    36
c     9
d    20
e    44
dtype: int64
total sum:  123


In [20]:
# Creating subsets

import itertools
subsets = []
stuff = item_list
for L in range(0, len(stuff)+1):
    for subset in itertools.combinations(stuff, L):
        subsets.append(list(subset))

#subsets

In [21]:
# finding the row wise items (have to optimise the step)

col_list_full = []
for index, row in basket.iterrows():
    col_list = []
    for col in basket:
            #if col in 
        if(basket.at[index,col]!=0.0):
            col_list.append(col)
    col_list_full.append(col_list)
    #print('-------------------------------------------------------------------------')
            #print(round(row.sum(),2))
#col_list_full   

In [22]:
# grouping the invoice and columns based on the subsets

before_grp = []
for i in range (len(basket)):
    temp = [str(col_list_full[i]),invoice_sum[i]]
    #print(col_list_full[i], ' ',invoice_sum[i])
    before_grp.append(temp)

before_grp

e_dataframe = pd.DataFrame(before_grp)
e_dataframe.columns = ['a', 'b']

e_dataframe

groupped_sum = pd.DataFrame(e_dataframe.groupby(['a']).sum())
groupped_sum.reset_index(level=0, inplace=True)
groupped_sum.columns = ['a', 'b']

In [23]:
# finding characteristic function value for each group and assigning them to subset list
all_sums = []
for sub in subsets:
    sum1 = 0
    print('subset id:', subsets.index(sub))
    print("step1")
    #print(sub)
    print('------------')
    stuff = sub
    new_sub = []
    for L in range(1, len(stuff)+1):
        for subset in itertools.combinations(stuff, L):
            new_sub.append(list(subset))
    print(new_sub)
    for item in new_sub:
        index = groupped_sum[groupped_sum['a'] == str(item)].index
        if index.empty:
            print('None')
        else:
            print(item, index)
            sum1+=groupped_sum['b'][index].values[0]
    all_sums.append(sum1)
    print('------------------------------------------------------------')

subset id: 0
step1
------------
[]
------------------------------------------------------------
subset id: 1
step1
------------
[['a']]
None
------------------------------------------------------------
subset id: 2
step1
------------
[['b']]
['b'] Int64Index([7], dtype='int64')
------------------------------------------------------------
subset id: 3
step1
------------
[['c']]
None
------------------------------------------------------------
subset id: 4
step1
------------
[['d']]
None
------------------------------------------------------------
subset id: 5
step1
------------
[['e']]
None
------------------------------------------------------------
subset id: 6
step1
------------
[['a'], ['b'], ['a', 'b']]
None
['b'] Int64Index([7], dtype='int64')
['a', 'b'] Int64Index([1], dtype='int64')
------------------------------------------------------------
subset id: 7
step1
------------
[['a'], ['c'], ['a', 'c']]
None
None
None
------------------------------------------------------------
sub

In [24]:
all_sums

[0,
 0,
 12,
 0,
 0,
 0,
 22,
 0,
 0,
 6,
 12,
 27,
 12,
 0,
 0,
 0,
 22,
 37,
 28,
 0,
 6,
 14,
 27,
 28,
 27,
 28,
 37,
 54,
 51,
 60,
 71,
 123]

In [25]:
new_dict = {}
for i in range(len(all_sums)):
    new_dict.update({str(subsets[i]) : all_sums[i] })
    print(subsets[i], " ", all_sums[i])

[]   0
['a']   0
['b']   12
['c']   0
['d']   0
['e']   0
['a', 'b']   22
['a', 'c']   0
['a', 'd']   0
['a', 'e']   6
['b', 'c']   12
['b', 'd']   27
['b', 'e']   12
['c', 'd']   0
['c', 'e']   0
['d', 'e']   0
['a', 'b', 'c']   22
['a', 'b', 'd']   37
['a', 'b', 'e']   28
['a', 'c', 'd']   0
['a', 'c', 'e']   6
['a', 'd', 'e']   14
['b', 'c', 'd']   27
['b', 'c', 'e']   28
['b', 'd', 'e']   27
['c', 'd', 'e']   28
['a', 'b', 'c', 'd']   37
['a', 'b', 'c', 'e']   54
['a', 'b', 'd', 'e']   51
['a', 'c', 'd', 'e']   60
['b', 'c', 'd', 'e']   71
['a', 'b', 'c', 'd', 'e']   123


In [26]:
#new_dict

# Finding marginal Contribution

In [27]:
dict1 = {'subsets': subsets}
subsets_df = pd.DataFrame((dict1))
subsets_df

,subsets
0,[]
1,[a]
2,[b]
3,[c]
4,[d]
5,[e]
6,"[a, b]"
7,"[a, c]"
8,"[a, d]"
9,"[a, e]"


In [28]:
# adding items to the columns
struct1 = pd.concat([subsets_df,pd.DataFrame(columns=item_list)])
struct1 = struct1.fillna(0)
struct1

,subsets,a,b,c,d,e
0,[],0,0,0,0,0
1,[a],0,0,0,0,0
2,[b],0,0,0,0,0
3,[c],0,0,0,0,0
4,[d],0,0,0,0,0
5,[e],0,0,0,0,0
6,"[a, b]",0,0,0,0,0
7,"[a, c]",0,0,0,0,0
8,"[a, d]",0,0,0,0,0
9,"[a, e]",0,0,0,0,0


In [29]:
columns_list = item_list
print(len(columns_list))

index_list = subsets
len(index_list[6])

5


2

In [30]:
import math as m
def fact(n):
    return m.factorial(n)


In [31]:
## Take 2

count = 0
for index in index_list:
    print('========================================================================')
    print('item coalition (index): ', index)
    print('-----------------------')
    for col in columns_list:
        
        print('item (column) = ', col)
        if col in index:
            temp = index[:]
            
            temp.remove(col)
            print('marginal contribution = v()', index, ' - v() ', temp)
            print('marginal contribution = v()', new_dict[str(index)], ' - v() ',new_dict[str(temp)])
            struct1.loc[subsets.index(index),col] = new_dict[str(index)] - new_dict[str(temp)]
            idx_len = len(index)
            col_len = len(columns_list)
            struct1.loc[subsets.index(index),col] = struct1.loc[subsets.index(index),col] * ((fact(idx_len - 1)*fact(col_len-idx_len))/ fact(col_len))
            count+=1
            #print('item list = ', index, ' value of items ', temp)
            print('-----------------------')
            #print(index, list(str(col)))
        else:
            print('marginal contribution = 0')
            print('-----------------------')
            count+=1
            continue
        
        
        
            #temp.append(0)
    
    #print(temp)
    #new_struct.append(temp)
#

#pd.DataFrame(new_struct)


item coalition (index):  []
-----------------------
item (column) =  a
marginal contribution = 0
-----------------------
item (column) =  b
marginal contribution = 0
-----------------------
item (column) =  c
marginal contribution = 0
-----------------------
item (column) =  d
marginal contribution = 0
-----------------------
item (column) =  e
marginal contribution = 0
-----------------------
item coalition (index):  ['a']
-----------------------
item (column) =  a
marginal contribution = v() ['a']  - v()  []
marginal contribution = v() 0  - v()  0
-----------------------
item (column) =  b
marginal contribution = 0
-----------------------
item (column) =  c
marginal contribution = 0
-----------------------
item (column) =  d
marginal contribution = 0
-----------------------
item (column) =  e
marginal contribution = 0
-----------------------
item coalition (index):  ['b']
-----------------------
item (column) =  a
marginal contribution = 0
-----------------------
item (column) =  b
m

item (column) =  a
marginal contribution = 0
-----------------------
item (column) =  b
marginal contribution = v() ['b', 'c', 'd']  - v()  ['c', 'd']
marginal contribution = v() 27  - v()  0
-----------------------
item (column) =  c
marginal contribution = v() ['b', 'c', 'd']  - v()  ['b', 'd']
marginal contribution = v() 27  - v()  27
-----------------------
item (column) =  d
marginal contribution = v() ['b', 'c', 'd']  - v()  ['b', 'c']
marginal contribution = v() 27  - v()  12
-----------------------
item (column) =  e
marginal contribution = 0
-----------------------
item coalition (index):  ['b', 'c', 'e']
-----------------------
item (column) =  a
marginal contribution = 0
-----------------------
item (column) =  b
marginal contribution = v() ['b', 'c', 'e']  - v()  ['c', 'e']
marginal contribution = v() 28  - v()  0
-----------------------
item (column) =  c
marginal contribution = v() ['b', 'c', 'e']  - v()  ['b', 'e']
marginal contribution = v() 28  - v()  12
--------------

In [32]:
print('run count = ', count)
struct1

run count =  160


,subsets,a,b,c,d,e
0,[],0.000000,0.000000,0.000000,0.000000,0.000000
1,[a],0.000000,0.000000,0.000000,0.000000,0.000000
2,[b],0.000000,2.400000,0.000000,0.000000,0.000000
3,[c],0.000000,0.000000,0.000000,0.000000,0.000000
4,[d],0.000000,0.000000,0.000000,0.000000,0.000000
5,[e],0.000000,0.000000,0.000000,0.000000,0.000000
6,"[a, b]",0.500000,1.100000,0.000000,0.000000,0.000000
7,"[a, c]",0.000000,0.000000,0.000000,0.000000,0.000000
8,"[a, d]",0.000000,0.000000,0.000000,0.000000,0.000000
9,"[a, e]",0.300000,0.000000,0.000000,0.000000,0.300000


In [33]:
shapleys = struct1.sum()/quantity
shapleys

a          2.52381
b          2.69444
c          2.40741
d              2.4
e          2.48485
subsets        NaN
dtype: object

In [34]:
quantity

a     7
b    12
c     9
d    10
e    11
dtype: int64